In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 1. 데이터 불러오기
df = pd.read_csv("acoustic_fire.csv")
df['FUEL'] = LabelEncoder().fit_transform(df['FUEL'])
X = df[['SIZE', 'FUEL', 'DISTANCE', 'DESIBEL', 'AIRFLOW', 'FREQUENCY']].values
y = df['STATUS'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_acc = accuracy_score(y_test, rf.predict(X_test))

# 4. Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)

# 5. LSTM model
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(6, 64, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        return torch.sigmoid(self.fc(out[:, -1, :]))

lstm_model = LSTMModel()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
for epoch in range(10):
    for xb, yb in train_loader:
        pred = lstm_model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
lstm_model.eval()
lstm_acc = ((lstm_model(X_test_tensor) > 0.5).float() == y_test_tensor).float().mean().item()

# 6. CBAM + CNN
class CBAM(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.channel = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Conv1d(channels, channels//8, 1),
            nn.ReLU(),
            nn.Conv1d(channels//8, channels, 1),
            nn.Sigmoid()
        )
        self.spatial = nn.Sequential(
            nn.Conv1d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        ca = self.channel(x)
        x = x * ca
        sa_input = torch.cat([x.mean(1, keepdim=True), x.max(1, keepdim=True)[0]], dim=1)
        sa = self.spatial(sa_input)
        return x * sa

class CNN_CBAM(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 64, 3, padding=1),
            nn.ReLU(),
            CBAM(64),
            nn.Conv1d(64, 128, 3, padding=1),
            nn.ReLU()
        )
        self.fc = nn.Linear(128 * 6, 1)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return torch.sigmoid(self.fc(x))

cnn_cbam = CNN_CBAM()
optimizer = torch.optim.Adam(cnn_cbam.parameters(), lr=0.001)
for epoch in range(10):
    for xb, yb in train_loader:
        pred = cnn_cbam(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
cnn_cbam.eval()
cbam_acc = ((cnn_cbam(X_test_tensor) > 0.5).float() == y_test_tensor).float().mean().item()

# 7. PatchTST
class PatchTST(nn.Module):
    def __init__(self, input_dim=6, patch_size=2, d_model=64, num_heads=4):
        super().__init__()
        self.linear = nn.Linear(input_dim * patch_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads)
        self.encoder = nn.TransformerEncoder(encoder_layer, 2)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        B, D = x.shape
        x = x.view(B, 3, 2*D//6)  # 3 patches
        x = self.linear(x)
        x = self.encoder(x)
        x = x.mean(1)
        return torch.sigmoid(self.fc(x))

patchtst = PatchTST()
optimizer = torch.optim.Adam(patchtst.parameters(), lr=0.001)
for epoch in range(10):
    for xb, yb in train_loader:
        pred = patchtst(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
patchtst.eval()
patchtst_acc = ((patchtst(X_test_tensor) > 0.5).float() == y_test_tensor).float().mean().item()

# 8. 결과 출력
print(f"Random Forest Accuracy: {rf_acc:.4f}")
print(f"LSTM Accuracy:          {lstm_acc:.4f}")
print(f"CNN + CBAM Accuracy:    {cbam_acc:.4f}")
print(f"PatchTST Accuracy:      {patchtst_acc:.4f}")
